# Importar bibliotecas generales

In [ ]:
import numpy as np
import pandas as pd
import re as regex
import math as math

# Importar bibliotecas propias 

In [ ]:
#biblioteca para completar valores Nan de la columna rooms
%run "analize_description_title.ipynb"

# Leer el data set desde el archivo y generar el data frame

In [ ]:
pathArchivoDataSet = 'properatti.csv'
df = pd.read_csv(pathArchivoDataSet)

# Detalle generales del data frame

In [ ]:
df.info()

In [ ]:
df.head()

In [ ]:
df.columns

# La primer columna no tiene un nombre asignado

In [ ]:
type(df.iloc[0,0])

In [ ]:
len(df.loc[df.index == df['Unnamed: 0']])

## Para todas las filas del data frame, el valor de la primer columna es igual al valor del index. Se puede asumir que dicha columna corresponde a un campo ID. Se renombra la primer columna

In [ ]:
df.rename(columns={'Unnamed: 0': 'Id'}, inplace = True)
df.columns

# Limpieza del data set 

### Se eliminan columnas innecesarias

In [ ]:
#quitamos la columna floor, ya que es difícil de inferir a partir de las demás columnas, por simplicidad se quita de los datos
df.drop(labels='floor', axis = 1, inplace = True)

In [ ]:
#quitamos la columna image_thumbnail ya que no aporta información relevante
df = df.drop(labels='image_thumbnail', axis= 1)

In [ ]:
#quitamos la columna properati_url ya que no aporta información relevante
df = df.drop(labels='properati_url', axis= 1)

In [ ]:
#chequeamos las columnas que nos quedan
df.columns

### Se eliminan datos (filas) que no pudieron ser completados

In [ ]:
#se eliminan filas cuyo campo description está vacío, dicha columna se tomó como fuente para otras columnas
#al venir vacía otros datos no pueden ser completados
emptyDescriptionIndexes = df[df['description'].isnull()].index

#borramos las filas con los correspondientes indices
df.drop(emptyDescriptionIndexes , inplace=True)

#reseteamos los índices para poder seguir usándolos sin problemas
df.reset_index();

# Completar valores faltantes

### A partir del dato geonames_id, generamos tres columnas nuevas, que corresponden a la coordenada representativa de cada zona (representada por un geonames_id). Esta coordenada no corresponde a la de la propiedad

##### Leemos directamente el archivo CSV generado en la notebook auxiliar "Completar coordenadas desde geonames_id" . Para ver el proceso remitirse a dicha notebook

In [ ]:
latLngCSVFileName = 'latLngFromGeonames.csv'
#cargamos el archivo
latLongDF = pd.read_csv(latLngCSVFileName)

In [ ]:
#mergeamos el data frame leído con el da properatti usando el dato geonames_id
df = df.merge(latLongDF, how='left', left_on='geonames_id', right_on='geonames_id', suffixes=('', '_geonames'))

### Datos de coordenadas de propiedades

#### Armamos 2 DataFrame:
    (A) 1 DataFrame para Ordenada Latitud: de columnas 'place_name' y 'lat'
    (B) 1 DataFrame para Ordenada Longitud: de columnas 'place_name' y 'lon'

(A) DF de Ordenada Latitud

In [ ]:
# Definimos el subset del dataframe 'data' para quedarnos sólo con las columnas 'place_name' y 'lat'
# una vez definida se pasa el método ".head()" que despliega las primeras 5 filas del Dataframe 

dflatitud=df[['place_name', 'lat']]
dflatitud.head()

Comprobamos cantidad de filas completas de la columna 'lat' en comparación al total de filas

In [ ]:
dflatitud.info()

De (A) nos quedamos solo con los campos con datos, de la columna 'lat', desechando los valores 'NaN'

In [ ]:
dflatitud_1=dflatitud.dropna()
dflatitud_1.head()

Volvemos a tomar cantidad de datos por fila y vemos que se eliminaron los NaN del Dataframe.
Observamos también, que en 'lat' bajaron 23 filas más, lo que indica que en 'place_name' también eran NaN. Se eliminan son sólo 23 en 121220 (0,018%)

In [ ]:
dflatitud_1.info()

Calulamos la media de las latitudes por grupo de localidad, con el método ".groupby()" P.ej.: toma todas las filas de localidad igual a "Abasto", de la columna 'place_name' y calcula la media de las Latitudes

In [ ]:
dflatitud_1=dflatitud_1.groupby(['place_name']).mean()
dflatitud_1.head()

Comprobamos que la cantidad de filas vuelve a disminuir, puesto que sólo muestra la media de localidades únicas.

In [ ]:
len(dflatitud_1)

(B) DF de Ordenada Longitud

###### Armo un subset sólo con las columnas 'place_name' y 'lon'

In [ ]:
# Definimos el subset del dataframe 'data' para quedarnos sólo con las columnas 'place_name' y 'lat'
# una vez definida se pasa el método ".head()" que despliega las primeras 5 filas del Dataframe 
dflongitud=df[['place_name', 'lon']]
dflongitud.head()

Comprobamos cantidad de filas completas de la columna 'lon' en comparación al total de filas

In [ ]:
dflongitud.info()

De (B) nos quedamos solo con los campos con datos, de la columna 'lon', desechando los valores 'NaN'

In [ ]:
dflongitud_1=dflongitud.dropna()
dflongitud_1.head()

Volvemos a tomar cantidad de datos por fila y vemos que se eliminaron los NaN del Dataframe.
Observamos también, que en 'lon' bajaron 23 filas más, lo que indica que en 'place_name' también eran NaN. Se eliminan son sólo 23 en 121220 (0,018%)

In [ ]:
dflongitud_1.info()

Calulamos la media de las longtudes por grupo de localidad, con el método ".groupby()" P.ej.: toma todas las filas de localidad igual a "Abasto", de la columna 'place_name' y calcula la media de las Longitudes

In [ ]:
dflongitud_1=dflongitud_1.groupby(['place_name']).mean()
dflongitud_1.head()

Comprobamos que la cantidad de filas vuelve a disminuir, puesto que sólo muestra la media de localidades únicas.

In [ ]:
len(dflongitud_1)

Definimos un nuevo dataframe que una las columnas lat y lon de los puntos (A) y (B).
Para ello vamos utilizar el método ".merge()" de Pandas

In [ ]:
dflatlon=pd.merge(dflatitud_1, dflongitud_1, left_on='place_name', right_on='place_name')
dflatlon

Tomamos al nuevo dataframe y generamos un nuevo índice para que 'place_name' vuelva a ser una variable del df

In [ ]:
dflatlon.reset_index(inplace=True)
dflatlon

Tomamos al dataframe original (df) y hacemos un Join con Pandas, sumando al final del DF las columnas con las medias de 'lat' y 'lon'

In [ ]:
df = df.merge(dflatlon, how='left', left_on='place_name', right_on='place_name', suffixes=('','_mediaPorZona'))
df.head(5)

### Datos de localidades: place_with_parent_names, place_name, country_name, state_name

In [ ]:
df[['place_with_parent_names', 'place_name','country_name','state_name']].isnull().sum()

##### Solo faltan 23 valores en la columna 'place_name' (información de barrio, zona, ciudad, etc.) que se intentarán obtener del la columna 'place_with_parent_names'.

In [ ]:
#seteamos la longitud del output para mejor lectura
pd.set_option('display.max_colwidth', -1)

##### Tomamos las columnas necesarias para completar los faltantes

In [ ]:
df[['place_with_parent_names', 'place_name','country_name','state_name']]

##### Se crea una nueva columna con los valores de la columna 'place_with_parent_names' en forma de lista

In [ ]:
def disgrega(valor):
    return valor.strip('|').split('|')

df['lista_auxiliar'] = df['place_with_parent_names'].apply(lambda x: disgrega(x))

In [ ]:
df['lista_auxiliar']

##### Se calcula cuantos elementos tiene cada lista

In [ ]:
df['conteo'] = df['lista_auxiliar'].apply(lambda x: len(x))
df['conteo']

##### Se determina cual es el mayor y menor números de elemntos en un registro de 'place_with_parent_names'

In [ ]:
df['conteo'].max()

In [ ]:
df['conteo'].min()

##### Se necesitarán 5 columnas como máximo para desempacar las distintas jerarquías de las locaciones. Mediante el comando apply se obtienen 5 columnas extras con información de localización por regiones.

In [ ]:
def chequeo2(valor):
    if len(valor) > 2:
        return valor[2]
    
    
def chequeo3(valor):
    if len(valor) > 3:
        return valor[3]
    
def chequeo4(valor):
    if len(valor) > 4:
        return valor[4]

df['loc1'] = df['lista_auxiliar'].apply(lambda x: x[0])
df['loc2'] = df['lista_auxiliar'].apply(lambda x: x[1])
df['loc3'] = df['lista_auxiliar'].apply(lambda x: chequeo2(x))
df['loc4'] = df['lista_auxiliar'].apply(lambda x: chequeo3(x))
df['loc5'] = df['lista_auxiliar'].apply(lambda x: chequeo4(x))

In [ ]:
daux = df[(df.loc5.notnull())]
daux[['loc1', 'loc2', 'loc3', 'loc4', 'loc5']]

##### Verificamos que los registros que tenían place_name vacíos tengan valores válidos en las nuevas columnas para cubrir el faltante

In [ ]:
len(df[df['place_name'].isna() & ~df['loc5'].isna()])

In [ ]:
df[df['place_name'].isna()][['lat','lon','geonames_id','place_name','place_with_parent_names', 'loc1', 'loc2', 'loc3', 'loc4', 'loc5']]

##### Property_Type => OK

In [ ]:
#Vemos los tipos de propiedades existentes, y su correspondiente cantidad

df['property_type'].value_counts()

In [ ]:
#Revisamos si alguna linea tiene el tipo de propiedad vacio
#Vemos que no, que todas las 121.220 lineas existentes tienen el campo completo

df['property_type'].value_counts().sum()

In [ ]:
house = df.loc[:, 'property_type'] == 'house'
print("La cantidad de house es: ",house.sum())

apartment = df.loc[:, 'property_type'] == 'apartment'
print("La cantidad de apartment es: ",apartment.sum())

PH = df.loc[:, 'property_type'] == 'PH'
print("La cantidad de PH es: ",PH.sum())

store = df.loc[:, 'property_type'] == 'store'
print("La cantidad de store es: ",store.sum())

tipodepropiedad = house & apartment & PH & store
print("La cantidad de propiedades sin tipo es: ",tipodepropiedad.sum())

##### Surface (covered y total)

###### Opcion 1: funcion para obtener metros cuadrados (a partir de surface_total_in_m2 y surface_covered_in_m2)

In [ ]:
print('Cantidad registros surface_covered_in_m2 sin valor: ' + str(df['surface_covered_in_m2'].isnull().sum()))
print('Cantidad registros surface_total_in_m2 sin valor: ' + str(df['surface_total_in_m2'].isnull().sum()))

###### Vamos a tratar de llenar los datos faltantes de superficie total en el dataset, rellenando con el promedio por barrio de cada publicacion

In [ ]:
#Primero vamos a ordenar el dataset para poder tener los partidos en orden y limpios
df.rename(index=str, inplace=True)

In [ ]:
#Primero hacmemos data wrangling de la columna "place_with_parent_names"

df.place_with_parent_names = df.place_with_parent_names.map(str.lower) #llevo a minusculas para evitar duplicados

grouped_places = df.groupby(['place_with_parent_names']) #agrupo por place_with_parent_names

dictio_places = grouped_places.groups.keys() #genero diccionario de places

cantidad_places = len(dictio_places) #cuento la cantidad de places distintos

print("Cantidad de place_with_parent_names distintos: ",cantidad_places)

In [ ]:
count_x_places = grouped_places.agg({"Id": "count"}) #agrupo y cuento

count_x_places = count_x_places.rename(index=str, columns={"Id": "cantidad"}) #renombro la columna por cantidad

count_x_places = count_x_places.sort_values(by=['cantidad'], ascending=False) #ordeno por cantidad descendente

In [ ]:
len(count_x_places.query("cantidad > 50")) # places con mas de <n> registros. Solo consulta

In [ ]:
list_places = [sub_places.split('|') for sub_places in count_x_places.index]

df_places = pd.DataFrame(list_places, 
                         index = count_x_places.index, 
                         columns =['none1','pais','provincia','partido','localidad','barrio','none2'])

df_places = df_places.drop(['none1', 'none2'], axis=1) # elimino none1 y none2
df_places = df_places.drop(['pais'], axis=1) # elimino pais ya que todos son argentina
df_places[df_places.barrio.notnull()] # el unico que tiene barrios es tigre-nordelta
df_places = df_places.drop(['barrio'], axis=1) # elimino barrio tambien

In [ ]:
# Para cada columna busco vacios y asigno None

for column_name in df_places.columns:
    df_places[column_name][df_places[column_name].apply(lambda column_name: True if regex.search('^\s*$', str(column_name)) else False)]=None

df_places = df_places.sort_values(by=['provincia','partido','localidad']) #ordeno

# creo df de provincias partidos y localidades
df_provincias = pd.DataFrame(df_places.provincia.unique(),columns=['nombre'])
df_partidos = pd.DataFrame(df_places.partido.unique(),columns=['nombre'])
df_localidades = pd.DataFrame(df_places.localidad.unique(),columns=['nombre'])

def buscar_reemplazar_place_column(row,column_name,df_base):
    if row[column_name]:
        idx = df_base.index[df_base.nombre == row[column_name]]
        return int(idx.data[0])
    
df_places.provincia =  df_places.apply(buscar_reemplazar_place_column,args=('provincia',df_provincias),axis=1)
df_places.partido =  df_places.apply(buscar_reemplazar_place_column,args=('partido',df_partidos),axis=1)
df_places.localidad =  df_places.apply(buscar_reemplazar_place_column,args=('localidad',df_localidades),axis=1)
df_places.sample(5)

In [ ]:
# agrega la relacion para las columnas que se vayan pasando respecto al dataframe de provincias, localidades y partidos

def agregar_columna_place(row,column_name,test):
    if (row.place_with_parent_names):
        return df_places.loc[row.place_with_parent_names][column_name]
    
provincias = df.apply(agregar_columna_place,args=('provincia','random'),axis=1)
df['provincia'] = pd.Series(provincias, index=df.index)
partidos = df.apply(agregar_columna_place,args=('partido','random'),axis=1)
df['partido'] = pd.Series(partidos, index=df.index)
localidades = df.apply(agregar_columna_place,args=('localidad','random'),axis=1)
df['localidad'] = pd.Series(localidades, index=df.index)
df.sample(3)

In [ ]:
df['state_name'].value_counts()

In [ ]:
df['provincia'].value_counts()

In [ ]:
# Analisis del cambio de columnas realizado con place_with parent names

df.place_name = df.place_name[df.place_name.notnull()].map(str.lower)
df_place_name = df.place_name

df_place_name_not_in = df_place_name[~(df_place_name.isin(df_partidos.nombre))]
df_place_name_not_in_loc = df_place_name_not_in[~(df_place_name_not_in.isin(df_localidades.nombre))]
df_place_name_not_in_loc_prov = df_place_name_not_in_loc[~(df_place_name_not_in_loc.isin(df_provincias.nombre))]
df_place_name_not_in_loc_prov.unique()
#Puedo eliminar la columna place_name

In [ ]:
# Reemplazo los valores que encuentro en place_name y que estan definidos en partidos

df.place_name = df.place_name[df.place_name.notnull()].map(str.lower)

def buscar_reemplazar_place_definidos(row,column_name,df_base): # funcion que buscar y reemplaza de la columna base (provincia, localidad, partido)
    a = df_base.nombre.str.contains(row[column_name], regex=False).any()
    if a:
        idx = df_base.index[df_base.nombre == row[column_name]]
        return int(idx.data[0])
    
df_place_name = df.place_name
mask_in_partidos = (df_place_name.isin(df_partidos.nombre))
mask_not_column_partido = df.partido.isnull()

print("Partidos con null:" + str(df.partido.isnull().sum()))

In [ ]:
df_reemplazar_part = df[mask_not_column_partido&mask_in_partidos].apply(buscar_reemplazar_place_definidos,args=('place_name',df_partidos),axis=1)
df.partido.update(df_reemplazar_part)
print("Partidos con null luego de procesar:" + str(df.partido.isnull().sum()))

In [ ]:
# Reemplazo los valores que encuentro en place_name y que estan definidos en localidades

df_place_name = df.place_name
mask_in_localidades = (df_place_name.isin(df_localidades.nombre))
mask_not_column_localidad = df.localidad.isnull()

print("Localidades con null: " + str(df.localidad.isna().sum()))

df_reemplazar_loc = df[mask_not_column_localidad&mask_in_localidades].apply(buscar_reemplazar_place_definidos,args=('place_name',df_localidades),axis=1)
df.localidad.update(df_reemplazar_loc)

print("Localidades con null luego de procesar: " + str(df.localidad.isna().sum()))

###### Ahora vamos a limpiar la columna surface

In [ ]:
df_title = df[df.title.notnull()]
df_title.title = df_title.title.map(str.lower)
pattern_m2 = regex.compile("(\d+\s*) m2")

def get_m2(row):
    result = pattern_m2.search(row)
    try:
        str_aux = result.group(1)
        array_m2 = str_aux.split()
        m2 = array_m2[-1]
        try:
            m2 = float(m2)
            return m2
        except:
            return np.nan;
    except:
        return np.nan

print("Cantidad de nulos m2 total surface: ",df.surface_total_in_m2.isnull().sum())

m2_from_title = df_title.title.apply(get_m2)
m2_from_title[m2_from_title.notnull()]
df.surface_total_in_m2.update(m2_from_title)

print("Cantidad de nulos luego de procesar: ",df.surface_total_in_m2.isnull().sum())

In [ ]:
#Vamos a generar los 39.328 datos faltantes en surface_total en la tabla, y lo vamos a hacer
#rellenando con el promedio del barrio de cada propiedad

print("Cantidad de nulos en surface_total_in_m2 antes:",df.surface_total_in_m2.isnull().sum())

# creamos una proporción de metros cubiertos sobre metros totales - no puede ser mayor a 1!
propcubierto = df.surface_covered_in_m2 / df.surface_total_in_m2
mask = propcubierto < 1
propcubierto_clean  = propcubierto[mask]

In [ ]:
#Relevamos algunos casos anómalos, donde la proporción da mayor a uno (inverosímil). Desestimamos estos 1200 registros para este procedimiento.
tempmask = propcubierto > 1
np.sum(tempmask)

In [ ]:
#Creamos una nueva variable sin esos datos anómalos
mask = propcubierto < 1
propcubierto_clean  = propcubierto[mask]

In [ ]:
#Hay dos casos anómalos donde superficie cubierta es cero, los reemplazamos por np.nan para evitar conflictos en el siguiente paso.
masksurface0 = (df.surface_covered_in_m2 == 0)
df.surface_covered_in_m2[masksurface0].fillna(np.nan)

#Agregamos la columna al dataframe
df['propcubierto']=propcubierto_clean

#Agrupamos por provincia(partido) el porcentaje promedio de m2cubierto/m2total
avg_propcubiertobarrio = df.groupby('partido')["propcubierto"].mean().sort_values(ascending = False)

#Cantidad de datos para calcular la proporcion 
avg_propcubiertobarriocount = df.groupby('partido')["propcubierto"].count().sort_values(ascending = False)

#Condición de la regla 1
removerporcantidadmask = avg_propcubiertobarriocount > 30

#Cantidad de datos existentes en tabla
datos_en_tabla = df.groupby('partido')['partido'].count().sort_values(ascending = False)

#Divido los datos existentes sobre los datos totales y obtengo la relación para la regla DOS
proporcion = avg_propcubiertobarriocount / datos_en_tabla
proporcion.round(2).sort_values(ascending = True)

#Condición de la regla 2
removerporproporcionmask = proporcion > 0.25

#Genero máscara con ambas condiciones
proporcionmask2 = removerporcantidadmask & removerporproporcionmask
propvalidados = avg_propcubiertobarrio[proporcionmask2]

#Ahora iteramos por las propiedades que tienen el dato faltante de superficie total y les inputamos la proporción promedio
#del barrio al que pertenecen, usando como dato la superficie cubierta.
# SUPERFICIE TOTAL = (SUPERFICIE CUBIERTA / PROPORCION CUBIERTO TOTAL)

surface_total_in_m2_clean = []
for index, row in df.iterrows():
    
    if pd.isnull(row.surface_total_in_m2):    
        if(row.partido in propvalidados.index):        
            surface_total_in_m2_clean.append(row.surface_covered_in_m2 / propvalidados.loc[row.partido]) #VERSION CON PROPVALIDADOS NO CORRE
        else:
            surface_total_in_m2_clean.append(row.surface_total_in_m2)    
    else:
        surface_total_in_m2_clean.append(row.surface_total_in_m2)
df["surface_total_in_m2_nueva"] = surface_total_in_m2_clean
df["surface_total_in_m2_nueva"]

#dropeamos los casos NaN
surfacetotalnueva_condatos = df.surface_total_in_m2_nueva.dropna()

#y finalmente, los reemplazamos en la columna "surface_total_in_m2_nueva" original del DF
df.surface_total_in_m2.update(surfacetotalnueva_condatos)

print("Cantidad de nulos en surface_total_in_m2 despues:",df.surface_total_in_m2.isnull().sum())

In [ ]:
df.columns

In [ ]:
df.surface_total_in_m2.isnull().sum()

##### Completar el valor del campo rooms

In [ ]:
def completarValoresFaltantesEnFila(dataFrameRow):
    
    ##############################################################
    ##Actualizacion de Rooms utilizando el método definido en la notebbok auxiliar fill_column_rooms (entrega 1).ipynb
    updatedDataFameRow = updateRoomsFromRowData(dataFrameRow)
        
    return updatedDataFameRow

In [ ]:
df = df.apply(completarValoresFaltantesEnFila, axis=1);

###### Datos en las columnas description y title

In [ ]:
print('Cantidad de registros description con valor nulo: ' + str(df.description.isna().sum()))
print('Cantidad de registros title con valor nulo: ' + str(df.title.isna().sum()))

###### Columna price_usd_per_m2

In [ ]:
#definimos un diccionario con valores de conversion entre monedas locales y el USD
conversion_USD_a_monedas_locales = { 'ARS': 63, 'PEN': 3.53, 'UYU': 43.41, 'USD': 1}

In [ ]:
#funcion que recibe una fila del data frame y un diccionario con las conversiones entre monedas locales y el USD
#este metodo intenta calcular el precio por m2 en USD a partir de otras columnas del data frame
def updatePriceUSDPerM2(dataFrameRow, dolarConversion):
    
    conversion = 1
    
    #si el dato currency no tiene valor, asumimos que los valores son en USD (conversion = 1)
    #si currency tiene un valor, obtenemos el valor de conversion desde el diccionario dolarConversion
    if(not pd.isnull(dataFrameRow.currency)):
        conversion = dolarConversion[dataFrameRow.currency]
    
    #completamos solamente si la columna no tiene valor
    if(math.isnan(dataFrameRow.price_usd_per_m2)):
        #precio por m2 USD= precio total USD / superficie total
        if(not math.isnan(dataFrameRow.price_aprox_usd) and not math.isnan(dataFrameRow.surface_total_in_m2) and dataFrameRow.surface_total_in_m2 != 0):
            dataFrameRow.price_usd_per_m2 = dataFrameRow.price_aprox_usd / dataFrameRow.surface_total_in_m2
        
        #precio por m2 USD = (precio total $ * conversion moneda) / superficie total        
        if(math.isnan(dataFrameRow.price_usd_per_m2) 
           and not math.isnan(dataFrameRow.price) and not math.isnan(dataFrameRow.surface_total_in_m2) and dataFrameRow.surface_total_in_m2 != 0):
            dataFrameRow.price_usd_per_m2 = (dataFrameRow.price * conversion) / dataFrameRow.surface_total_in_m2
    
        #precio por m2 USD = precio por m2 * conversion moneda
        if(math.isnan(dataFrameRow.price_usd_per_m2) 
           and not math.isnan(dataFrameRow.price_per_m2)):
            dataFrameRow.price_usd_per_m2 = dataFrameRow.price_per_m2 * conversion
            
        #precio por m2 USD = precio aprox por m2 USD / superficie total
        if(math.isnan(dataFrameRow.price_usd_per_m2) 
           and not math.isnan(dataFrameRow.price_aprox_usd) and not math.isnan(dataFrameRow.surface_total_in_m2) and dataFrameRow.surface_total_in_m2 != 0):
            dataFrameRow.price_usd_per_m2 = dataFrameRow.price_aprox_usd / dataFrameRow.surface_total_in_m2
            
        #precio por m2 USD = precio aprox por m2 local currency * conversion moenda / superficie total
        if(math.isnan(dataFrameRow.price_usd_per_m2) 
           and not math.isnan(dataFrameRow.price_aprox_local_currency) and not math.isnan(dataFrameRow.surface_total_in_m2) and dataFrameRow.surface_total_in_m2 != 0):
            dataFrameRow.price_usd_per_m2 = (dataFrameRow.price_aprox_local_currency * conversion) / dataFrameRow.surface_total_in_m2
            
    return dataFrameRow

In [ ]:
#contamos price per m2 en USD antes de la ejecucion
df['price_usd_per_m2'].isna().sum()

In [ ]:
#aplicamos la funcion que actualiza los precios por m2 al data frame
df = df.apply(updatePriceUSDPerM2, axis=1, dolarConversion=conversion_USD_a_monedas_locales);

In [ ]:
#verificamos la cantidad de vacíos luego de la ejecución
df['price_usd_per_m2'].isna().sum()

###### Columna price_per_m2

In [ ]:
#similar al calculo del m2 por USD, el calculo del m2 en moneda local usa otras columnas del data frame para obtener el valor
#recibe tambien un diccionario con la conversion de monedas locales a USD
def updatePricePerM2(dataFrameRow, dolarConversion):
            
    #si el dato currency no tiene valor, asumimos que el valor precio por m2 a guardar es en ARS
    conversion = 1 / dolarConversion['ARS'] #si el dolar esta a 65 ARS => el factor de conversión de USD a ARS es 1 / 65
    
    #si currency tiene un valor, obtenemos el valor de conversion desde el diccionario dolarConversion
    if(not pd.isnull(dataFrameRow.currency)):
        conversion = 1 / dolarConversion[dataFrameRow.currency]
    
    #completamos solamente si la columna no tiene valor
    if(math.isnan(dataFrameRow.price_per_m2)):
        #precio por m2 = precio aprox total local currency / superficie total
        if(not math.isnan(dataFrameRow.price_aprox_local_currency) and not math.isnan(dataFrameRow.surface_total_in_m2) and dataFrameRow.surface_total_in_m2 != 0):
            dataFrameRow.price_usd_per_m2 = dataFrameRow.price_aprox_local_currency / dataFrameRow.surface_total_in_m2
        
        #precio por m2 = precio por m2 USD / conversion moneda
        if(math.isnan(dataFrameRow.price_per_m2) 
           and not math.isnan(dataFrameRow.price_usd_per_m2)):
            dataFrameRow.price_per_m2 = dataFrameRow.price_usd_per_m2 / conversion
            
        #precio por m2 USD = (precio aprox por m2 USD / conversion) / superficie total
        if(math.isnan(dataFrameRow.price_per_m2) 
           and not math.isnan(dataFrameRow.price_aprox_usd) and not math.isnan(dataFrameRow.surface_total_in_m2) and dataFrameRow.surface_total_in_m2 != 0):
            dataFrameRow.price_per_m2 = (dataFrameRow.price_aprox_usd / conversion) / dataFrameRow.surface_total_in_m2
             
    return dataFrameRow

In [ ]:
#contamos price per m2 en USD antes de la ejecucion
df['price_per_m2'].isna().sum()

In [ ]:
#aplicamos la funcion que actualiza los precios por m2 al data frame
df = df.apply(updatePricePerM2, axis=1, dolarConversion=conversion_USD_a_monedas_locales);

In [ ]:
#verificamos la cantidad de vacíos luego de la ejecución
df['price_per_m2'].isna().sum()

# Agregamos algunos datos de amenities analizando las columnas description y title

##### Los métodos que aplican las regex sobre las columnas description y title están definidos en la notebook auxiliar analize_description_title

In [ ]:
df['pileta'] = df.apply(lambda row: containsPool(row), axis = 1)
df['pileta'].value_counts()

In [ ]:
df['cochera'] = df.apply(lambda row: containsParking(row), axis = 1)
df['cochera'].value_counts()

In [ ]:
df['balcon'] = df.apply(lambda row: containsBalcony(row), axis = 1)
df['balcon'].value_counts()

In [ ]:
df['terraza'] = df.apply(lambda row: containsTerrace(row), axis = 1)
df['terraza'].value_counts()

In [ ]:
df['parrilla'] = df.apply(lambda row: containsGrill(row), axis = 1)
df['parrilla'].value_counts()